In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('plant_disease_data.csv')
df.head()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Vectorize symptom descriptions
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Symptom Description'])

# Encode disease labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Disease Name'])

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the classifier
model = MultinomialNB()
model.fit(X_train, y_train)


In [ ]:
def predict_disease(user_input):
    # Convert text to vector
    input_vec = vectorizer.transform([user_input])

    # Predict disease
    pred_label = model.predict(input_vec)[0]
    disease = label_encoder.inverse_transform([pred_label])[0]

    # Get solution from the dataframe
    solution = df[df['Disease Name'] == disease]['Solution'].values[0]

    return disease, solution


In [ ]:
while True:
    user_input = input("Describe your plant issue (or type 'exit'): ")
    if user_input.lower() == "exit":
        break
    disease, solution = predict_disease(user_input)
    print(f"\n🦠 Detected Disease: {disease}")
    print(f"💊 Suggested Treatment: {solution}\n")


In [ ]:
#!pip install streamlit pyngrok --quiet    => Run this in terminal to install dependencies


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
import socket

# Debugging info
st.set_page_config(page_title="Plant Doctor 🌱", layout="wide")
hostname = socket.gethostname()
local_ip = socket.gethostbyname(hostname)
st.sidebar.write(f"Running on: {local_ip}")

# Load dataset with error handling
@st.cache_data
def load_data():
    try:
        df = pd.read_csv("plant_disease_data.csv")
        df.columns = df.columns.str.strip()
        return df
    except Exception as e:
        st.error(f"Error loading data: {str(e)}")
        return None

df = load_data()

if df is not None:
    # Preprocess
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df['Symptom Description'])

    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(df['Disease Name'])

    model = MultinomialNB()
    model.fit(X, y)

    # Predict function
    def predict_disease(user_input):
        try:
            input_vec = vectorizer.transform([user_input])
            pred = model.predict(input_vec)[0]
            disease = label_encoder.inverse_transform([pred])[0]
            solution = df[df['Disease Name'] == disease]['Solution'].values[0]
            return disease, solution
        except Exception as e:
            return f"Error: {str(e)}", "Please try different symptoms"

    # UI
    st.title("🌿 Plant Disease Diagnosis")
    st.write("Describe your plant's symptoms below:")

    user_input = st.text_area("Examples: 'yellow leaves', 'white spots', 'wilting stems'")

    if st.button("Diagnose"):
        if user_input:
            with st.spinner("Analyzing symptoms..."):
                disease, solution = predict_disease(user_input)
                st.success(f"🔍 **Diagnosis:** {disease}")
                st.info(f"💊 **Treatment:** {solution}")
        else:
            st.warning("Please describe your plant's symptoms")
else:
    st.error("Failed to load disease database. Please check your data file.")

In [ ]:
from google.colab import files
uploaded = files.upload()


#here upload the data set in the form of csv file

In [ ]:
# Remove any existing ngrok/streamlit processes
# !pkill -f ngrok || true
# !pkill -f streamlit || true

# Install required packages
# !pip install -q pyngrok streamlit

# Set up ngrok (replace with your actual token from https://dashboard.ngrok.com/)
from pyngrok import ngrok
ngrok.set_auth_token("2ygQREvCeSPRnukSkXKFC7D3wJj_6bj33P2FapJRTCb4Qn9yL")  # REPLACE THIS

# Start Streamlit in background
#!streamlit run app.py --server.port=8501 --server.address=0.0.0.0 &>/dev/null &

# Create public URL
public_url = ngrok.connect(8501, bind_tls=True)
print("🌐 Public URL:", public_url.public_url)

In [ ]:
# !pkill streamlit          =>   when error occurs at public url run this and restart the session


# RUn all these in the terminal 
# !pkill -f ngrok || true
# !pkill -f streamlit || true
# !pip install -q pyngrok streamlit
#!streamlit run app.py --server.port=8501 --server.address=0.0.0.0 &>/dev/null &